# DeepSeek-R1-7B 模型推理代码详解

## 文件概述
这个文件展示了如何使用 **DeepSeek-R1-Distill-Qwen-7B** 模型进行文本生成。这是一个完整的LLM推理示例，涵盖从模型加载到生成输出的全过程。

## 1️⃣ 导入和加载模型

首先，我们需要从ModelScope导入必要的类并加载模型和分词器。

In [ ]:
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name = "/root/autodl-tmp/models/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="cuda"  # auto
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

### 代码说明

| 参数 | 说明 |
|------|------|
| `AutoModelForCausalLM` | 因果语言模型，用于文本生成任务 |
| `AutoTokenizer` | 分词器，将文本转换为token ID |
| `torch_dtype="auto"` | 自动选择合适的数据类型（float16或float32） |
| `device_map="cuda"` | 将模型部署到GPU上运行 |

### 关键概念

- **因果语言建模**：只能看到前面的token，用于生成式任务
- **分词器**：将文本和token ID相互转换的工具

## 2️⃣ 构建聊天消息

按照 OpenAI 聊天格式构建消息列表，定义系统提示和用户输入。

In [ ]:
prompt = "帮我写一个二分查找法"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]

print("消息格式：")
for msg in messages:
    print(f"  {msg['role']}: {msg['content']}")

### 消息格式的优势

- ✅ 清晰区分不同角色的内容
- ✅ 便于多轮对话管理
- ✅ 符合主流LLM的输入格式
- ✅ 易于扩展和维护

## 3️⃣ 应用聊天模板

将结构化消息转换为模型期望的格式。

In [ ]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

print("格式化后的文本：")
print(text)
print("\n" + "="*60)

### 参数说明

| 参数 | 说明 |
|------|------|
| `messages` | 输入的消息列表 |
| `tokenize=False` | 返回格式化后的文本字符串（而非token ID） |
| `add_generation_prompt=True` | 添加生成提示符，告诉模型开始生成文本 |

### 转换效果示例

```
原始格式：
[{"role": "system", ...}, {"role": "user", ...}]

转换后：
<|im_start|>system
You are a helpful assistant.
<|im_end|>
<|im_start|>user
帮我写一个二分查找法
<|im_end|>
<|im_start|>assistant
```

## 4️⃣ 准备模型输入

将文本分词并转换为PyTorch张量，准备输入模型。

In [ ]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

print("模型输入结构：")
print(f"  Input IDs shape: {model_inputs['input_ids'].shape}")
print(f"  Attention Mask shape: {model_inputs['attention_mask'].shape}")
print(f"\n前10个token ID: {model_inputs['input_ids'][0][:10].tolist()}")

### 代码说明

- `tokenizer([text], ...)`：对文本进行分词
  - `[text]` 以列表形式输入，支持批处理
  - 返回token ID序列
- `return_tensors="pt"`：返回PyTorch张量格式
- `.to(model.device)`：将张量移到模型所在设备（GPU或CPU）

### 输出结构

```python
{
    'input_ids': tensor([[token_id_1, token_id_2, ...]]),
    'attention_mask': tensor([[1, 1, 1, ...]])  # 指示哪些位置是真实token
}
```

## 5️⃣ 文本生成

执行自回归文本生成。

In [ ]:
print("开始生成文本...（这可能需要几分钟）\n")

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2000
)

print(f"生成完成！")
print(f"生成的token总数: {generated_ids.shape[1]}")

### 生成原理

1. 🔹 输入初始token序列
2. 🔹 模型预测下一个最可能的token
3. 🔹 将预测的token加入序列
4. 🔹 重复步骤2-3，直到生成[EOS]（结束）token或达到max_new_tokens

### 参数说明

| 参数 | 说明 |
|------|------|
| `**model_inputs` | 解包输入字典作为关键字参数 |
| `max_new_tokens=2000` | 限制生成的最大token数 |

## 6️⃣ 提取生成部分

生成的output_ids包含输入部分和生成部分，我们只需要生成的新token。

In [ ]:
# 保留生成部分（移除输入部分）
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

print(f"输入token数: {len(model_inputs.input_ids[0])}")
print(f"生成token数: {len(generated_ids[0])}")

### 提取逻辑示例

```
output_ids:      [token_1, token_2, ..., token_100, token_101, ...]
                  └─输入部分─┘      └────生成部分────┘
len(input_ids) = 100

结果：[token_101, token_102, ...]
```

使用列表推导式处理批次中的每个样本。

## 7️⃣ 解码和输出

将token ID序列转换回文本字符串。

In [ ]:
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("="*60)
print("模型生成结果：")
print("="*60)
print(response)

### 代码说明

- `batch_decode()`：将token ID序列转换回文本字符串
- `skip_special_tokens=True`：忽略特殊token（如[CLS]、[SEP]等）
- `[0]`：取批次中第一个样本的结果

### 输出示例

```python
def binary_search(arr, target):
    left, right = 0, len(arr) - 1
    while left <= right:
        mid = (left + right) // 2
        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    return -1
```

## 📊 完整流程图

```
┌─────────────┐
│  输入文本   │
└──────┬──────┘
       ↓
┌─────────────────┐
│  [模板应用]     │  → 格式化为模型期望的格式
└──────┬──────────┘
       ↓
┌─────────────────┐
│  [分词]         │  → 转换为token ID序列
└──────┬──────────┘
       ↓
┌─────────────────┐
│  [移到GPU]      │  → 准备模型输入
└──────┬──────────┘
       ↓
┌─────────────────┐
│  [生成]         │  → 逐token自回归生成
└──────┬──────────┘
       ↓
┌─────────────────┐
│  [提取新token]  │  → 移除输入部分
└──────┬──────────┘
       ↓
┌─────────────────┐
│  [解码]         │  → token ID 转回文本
└──────┬──────────┘
       ↓
┌─────────────┐
│  输出结果   │
└─────────────┘
```

## 🔑 关键概念总结

| 概念 | 说明 | 作用 |
|------|------|------|
| **AutoModelForCausalLM** | 因果语言模型 | 用于文本生成 |
| **Tokenizer** | 文本-token转换工具 | 编码和解码 |
| **Chat Template** | 聊天格式模板 | 统一输入格式 |
| **Device Mapping** | 设备映射 | 指定计算设备 |
| **Generation** | 文本生成 | 自回归生成 |
| **Batch Processing** | 批处理 | 提高效率 |

## ❓ 常见问题

### Q1: 为什么要使用chat_template？
**A:** 不同的模型可能有不同的输入格式。ChatTemplate提供了一个统一的接口，自动处理格式转换。

### Q2: max_new_tokens的作用是什么？
**A:** 限制生成的长度，防止无限生成或资源浪费。token数越多，生成耗时和显存消耗越大。

### Q3: 生成速度与什么因素相关？
**A:**
- 模型大小（参数数量）
- 硬件配置（GPU显存和性能）
- 生成长度（max_new_tokens）
- 使用的数据类型（float32 vs float16）

### Q4: 如何加快推理速度？
**A:**
- 使用量化版本（如int8、int4）
- 使用GPU而非CPU
- 批量处理多个请求
- 使用缓存机制（KV cache）
- 使用流式生成

## 🚀 扩展应用

### 批量生成示例

In [ ]:
# 批量生成多个请求
prompts = [
    "帮我写一个二分查找法",
    "解释什么是OOP",
    "如何优化Python代码"
]

messages_list = [
    [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    for prompt in prompts
]

print(f"已准备 {len(messages_list)} 个请求")
for i, msg in enumerate(messages_list, 1):
    print(f"  {i}. {msg[-1]['content']}")

### 流式生成示例

In [ ]:
# 使用TextIteratorStreamer实现流式输出
# 这样可以实时显示生成的文本，而不用等到全部生成完成

from transformers import TextIteratorStreamer
from threading import Thread

# streamer = TextIteratorStreamer(tokenizer)
# generation_kwargs = dict(**model_inputs, streamer=streamer, max_new_tokens=2000)

# thread = Thread(target=model.generate, kwargs=generation_kwargs)
# thread.start()

# for text in streamer:
#     print(text, end='', flush=True)

print("流式生成实现示例（已注释，可根据需要启用）")

## 📝 总结

这个notebook详细展示了：

1. ✅ **模型加载** - 从ModelScope加载预训练模型和分词器
2. ✅ **消息构建** - 按照标准格式构建聊天消息
3. ✅ **格式转换** - 使用ChatTemplate统一输入格式
4. ✅ **分词处理** - 将文本转换为模型可接受的张量
5. ✅ **文本生成** - 自回归生成文本
6. ✅ **结果处理** - 提取和解码生成的文本

通过理解这个流程，你可以：n- 使用任何Hugging Face兼容的模型进行文本生成
- 构建自己的聊天应用
- 优化推理性能
- 实现更复杂的NLP任务

---

*文档最后更新：2026年2月4日*